In [ ]:
import numpy as np

In [6]:
cutoff = 4
potential = 'QHO'

In [7]:
q_coeff = np.sqrt(1/(2*cutoff))
q = q_coeff * np.diag(np.linspace(-cutoff + 1, cutoff - 1, cutoff))

In [8]:
q

array([[-1.06066017,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35355339,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.35355339,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.06066017]])

In [9]:
delta_q = 1 / q_coeff

# Initialize p^2 matrix
p2 = np.zeros((cutoff, cutoff))

# Fill the tridiagonal structure
factor = 1 / (delta_q**2)
for i in range(cutoff):
    p2[i, i] = -2 * factor  # Main diagonal
    if i > 0:
        p2[i, i - 1] = factor  # Lower diagonal
    if i < cutoff - 1:
        p2[i, i + 1] = factor  # Upper diagonal

In [10]:
b_cutoff = 4

# Calculate q^2 and q^3
q2 = np.matmul(q, q)
q3 = np.matmul(q2, q)

# Bosonic and fermionic identities
I_b = np.eye(b_cutoff)
I_f = np.eye(2)

# Superpotential derivatives
if potential == 'QHO':
    W_prime = q  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1

elif potential == 'AHO':
    W_prime = q + q3  # W'(q) = q + q^3
    W_double_prime = I_b + 3 * q2  # W''(q) = 1 + 3q^2

elif potential == 'DW':
    W_prime = q + q2 + I_b  # W'(q) = q + q^2 + 1
    W_double_prime = I_b + 2 * q  # W''(q) = 1 + 2q

else:
    raise ValueError("Not a valid potential")

# Kinetic term: p^2 (finite difference Laplacian)
#p2 = np.matmul(p, p)

# Commutator term [b^†, b] = -Z
Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
commutator_term = np.kron(Z, W_double_prime)

# Kinetic term (bosonic and fermionic parts)
kinetic_term = np.kron(I_f, p2)

# Potential term (W' contribution)
potential_term = np.kron(I_f, np.matmul(W_prime, W_prime))

# Construct the full Hamiltonian
H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)
H_SQM[np.abs(H_SQM) < 1e-12] = 0  # Eliminate small numerical artifacts


In [11]:
H_SQM

array([[ 0.9375,  0.0625,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 0.0625,  0.4375,  0.0625,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 0.    ,  0.0625,  0.4375,  0.0625,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 0.    ,  0.    ,  0.0625,  0.9375,  0.    ,  0.    ,  0.    ,
         0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    , -0.0625,  0.0625,  0.    ,
         0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.0625, -0.5625,  0.0625,
         0.    ],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.0625, -0.5625,
         0.0625],
       [ 0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.0625,
        -0.0625]])

In [14]:
eigenvalues[0]

-0.6318607642904032

In [ ]:
def calculate_Hamiltonian_fd(grid_points, x_min, x_max, potential):
    """
    Calculate the SUSY QM Hamiltonian using the finite difference method.
    
    Parameters:
    grid_points (int): Number of spatial grid points.
    x_min (float): Minimum value of the spatial domain.
    x_max (float): Maximum value of the spatial domain.
    potential (str): Type of potential ('QHO', 'AHO', 'DW').
    
    Returns:
    np.ndarray: The SUSY QM Hamiltonian matrix.
    """
    # Spatial grid and spacing
    x = np.linspace(x_min, x_max, grid_points)
    dx = x[1] - x[0]

    # Position matrix q (diagonal matrix)
    q = np.diag(x)

    # Momentum matrix p (finite difference approximation)
    prefactor = -1j / (2 * dx)  # Assuming hbar = 1
    p = np.zeros((grid_points, grid_points), dtype=np.complex128)
    for i in range(grid_points):
        if i > 0:
            p[i, i - 1] = prefactor
        if i < grid_points - 1:
            p[i, i + 1] = -prefactor

    # Calculate q^2 and q^3
    q2 = np.matmul(q, q)
    q3 = np.matmul(q2, q)

    # Bosonic and fermionic identities
    I_b = np.eye(grid_points)
    I_f = np.eye(2)

    # Superpotential derivatives
    if potential == 'QHO':
        W_prime = q  # W'(q) = q
        W_double_prime = I_b  # W''(q) = 1

    elif potential == 'AHO':
        W_prime = q + q3  # W'(q) = q + q^3
        W_double_prime = I_b + 3 * q2  # W''(q) = 1 + 3q^2

    elif potential == 'DW':
        W_prime = q + q2 + I_b  # W'(q) = q + q^2 + 1
        W_double_prime = I_b + 2 * q  # W''(q) = 1 + 2q

    else:
        raise ValueError("Not a valid potential")

    # Kinetic term: p^2 (finite difference Laplacian)
    p2 = np.matmul(p, p)

    # Commutator term [b^†, b] = -Z
    Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
    commutator_term = np.kron(Z, W_double_prime)

    # Kinetic term (bosonic and fermionic parts)
    kinetic_term = np.kron(I_f, p2)

    # Potential term (W' contribution)
    potential_term = np.kron(I_f, np.matmul(W_prime, W_prime))

    # Construct the full Hamiltonian
    H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)
    H_SQM[np.abs(H_SQM) < 1e-12] = 0  # Eliminate small numerical artifacts

    return H_SQM, x
